# Problemas da aula 7

Aqui estão minhas soluções para os problemas propostos na aula 7.

<!-- TEASER_END -->

In [ ]:
using PyPlot

## Problema 1

Encontre uma expressão para a segunda derivada numérica para pontos igualmente espaçados com espaçamento h

### Solução

$$
\frac{dy_{i+1/2}}{dx} \approx \frac{y_{i+1}-y_i}{h}\\
\frac{dy_{i-1/2}}{dx} \approx \frac{y_i - y_{i-1}}{h}\\
$$

$$
\frac{d^2y_i}{dx^2} \approx \frac{ \frac{dy_{i+1/2}}{dx} - \frac{dy_{i-1/2}}{dx}}{h} = \frac{y_{i+1} - 2y_i + y_{i-1}}{h^2}
$$



In [ ]:
function hseq(h₀, α, n)
    h1 = h₀
    
    h = zeros(n)
    h[1] = h1
    for i in 2:n
        h1 *= α
        h[i] = h1
    end
    
    return h
end


In [ ]:
d2fun(f, h) = x-> (f(x+h) - 2f(x) + f(x-h)) / h^2
d2fun(x, f, h) = (f(x+h) - 2f(x) + f(x-h)) / h^2
function d2fun2(x, f, h)
    yp1 = f(x+h)
    y = f(x)
    ym1 = f(x-h)
    
    return ( (yp1-y)/h + (ym1 - y)/h ) /h
end


In [ ]:
f(x) = sin(x)
df(x) = cos(x)
d2f(x) = -sin(x)
h = hseq(0.4, 0.8, 50);

In [ ]:
x0 = 3π/8

d2ye = d2f(x0)

d2y = d2fun.(x0, f, h)
err = abs.(d2y .- d2ye);


In [ ]:
loglog(h, err, "bo")
hh = 10.0 .^(-3:0.1:-1);
ee = hh.^2
plot(hh, ee, "r:")
text(1e-2, 1e-3, L"$\mathcal{O}(h^2)$")

In [ ]:
h = 0.00001
a1 = d2fun(x0, f, h)
a2 = d2fun2(x0, f, h)
a1 - d2ye, a2 - d2ye

## Problema 2

No notebook `07b-derivada-numerica.ipynb` foi construída um operador (uma matriz) para calcular as derivadas em todos os pontos. Use uma matriz tridiagonal para calcular a derivada $df/dx$ de uma função qualquer para $n$ nós igualmente espaçados entre os pontos a e b.



In [ ]:
using LinearAlgebra
using SparseArrays

In [ ]:
makex(a,b,n) = range(a,b,length=n+1)

function makeoperator(a, b, n)
    h = (b-a) / n
    ih = 1 / h
    
    d = zeros(n+1)
    d[1] = -ih
    d[end] = ih
    
    up = fill(0.5*ih, n)
    up[1] = ih
    
    lo = fill(-0.5*ih, n)
    lo[end] = -ih
    return Tridiagonal(lo, d, up)
end


In [ ]:
a = -π
b =  π
n = 20
x = makex(a, b, n)
D = makeoperator(a, b, n)

f(x) = sin(x)
df(x) = cos(x)
d2f(x) = -sin(x)
d3f(x) = -cos(x)
d4f(x) =  sin(x)

y = f.(x)
dye = df.(x)
d2ye = d2f.(x)
d3ye = d3f.(x);
d4ye = d4f.(x);

In [ ]:
dy = D * y
plot(x, dye, "ro")
plot(x, dy, "b.")

err = dy - dye;

## Problema 3

Use a matriz do problema anterior para calcular a segunda e terceira derivada. Compare com a solução analítica de alguma função simples


In [ ]:

d2y = D * dy;
d3y = D *  d2y;
d4y = D * d3y;

In [ ]:
plot(x, d2ye, "ro")
plot(x, d2y, "b.-")

In [ ]:
plot(x, d3ye, "ro")
plot(x, d3y, "b.-")

In [ ]:
plot(x, y, "ro")
plot(x, d4y, "b.-")

In [ ]:
Dm = Matrix(D)


In [ ]:
D2 = Dm * Dm
D2[7,:]

**Este esquema é inconsistente**


Apesar de parecer simples, este esqueme de simplesmente multiplicar operadores não funciona. Não é apenas que o erro é grande. O erro não converge para zero. Isto é muito comum em análise numérica!!!

Este esquema tem outro problema: a banda da matriz resultante é maior. Ao se resolver equações diferenciais usando diferenças finitas, a equação diferencial é substituída por um sistema linear. E quanto maior a largura de banda pior!!!

## Problema 4
Um dos problemas da matriz empregada no problema 2 é que nas extremidades, a derivada é aproximada por um esquema de 1a ordem. Deduza uma expressão de segunda ordem para as duas extremidades. Calcule a segunda e terceira derivada e compare os erros, inclusive no meio do domínio.

Dica: agora a matriz tridiagonal não será o suficiente. Pode-se usar matrizes cheias mas aí existe um desperdício de memória e tempo computacional. Uma alternativa é utilizar matrizes esparsas <https://docs.julialang.org/en/v1/stdlib/SparseArrays/>. Outra possibilidade melhor é utilizar matrizes em banda. Um pacote Julia para este tipo de matriz está disponível em <https://github.com/JuliaMatrices/BandedMatrices.jl>

### Solução

Usando os três primeiros nós para achar os polinômios de Lagrange, temos:

$$
h_1(x) = \frac{(x-x_2)(x-x_3)}{(x_1-x_2)(x_1-x_3)} = \frac{(x-x_2)(x-x_3)}{2h^2}\\
h_2(x) = \frac{(x-x_1)(x-x_3)}{(x_2-x_1)(x_2-x_3)} = -\frac{(x-x_1)(x-x_3)}{h^2}\\
h_3(x) = \frac{(x-x_1)(x-x_2)}{(x_3-x_1)(x_3-x_1)} = \frac{(x-x_1)(x-x_2)}{2h^2}\\
$$

Calculando a derivada no nó 1, temos

In [ ]:
using SymPy

@vars x x1 x2 x3 h

In [ ]:
H1  = (x-x2)*(x-x3)/( (x1-x2)*(x1-x3) )
H2  = (x-x1)*(x-x3)/( (x2-x1)*(x2-x3) )
H3  = (x-x1)*(x-x2)/( (x3-x1)*(x3-x2) )

dH1 = diff(H1, x)
dH2 = diff(H2, x)
dH3 = diff(H3, x);

In [ ]:
subs(dH1, x=>x1, x1=>0, x2=>h, x3=>2h)

In [ ]:
subs(dH2, x=>x1, x1=>0, x2=>h, x3=>2h)

In [ ]:
subs(dH3, x=>x1, x1=>0, x2=>h, x3=>2h)

In [ ]:
subs(dH1, x=>x3, x1=>0, x2=>h, x3=>2h)

In [ ]:
subs(dH2, x=>x3, x1=>0, x2=>h, x3=>2h)

In [ ]:
subs(dH3, x=>x3, x1=>0, x2=>h, x3=>2h)

In [ ]:
using BandedMatrices

In [ ]:
function makeoperator2(a, b, n)
    h = (b-a) / n
    
    D = BandedMatrix(Zeros(n+1, n+1), (2, 2))
    
    D[1,1] = -3/2h
    D[1,2] = 2/h
    D[1,3] = -1/2h
    
    D[end,end] = 3/2h
    D[end,end-1] = -2/h
    D[end,end-2] = 1/2h
    
    for i in 2:n
        D[i,i+1] = 1/2h
        D[i,i-1] = -1/2h
    end
    
    return D
    
end


In [ ]:
n = 20
D = makeoperator2(a, b, n)
x = makex(a,b,n)
y = f.(x)
dye = df.(x)
dy = D * y

plot(x, dye, "ro")
plot(x, dy, "b.")

In [ ]:
d2ye = d2f.(x)
d3ye = d3f.(x)
d4ye = d4f.(x)
d2y = D*dy
d3y = D*d2y
d4y = D*d3y;

In [ ]:
plot(x, d2ye, "ro")
plot(x, d2y, "b.")



In [ ]:
plot(x, d3ye, "ro")
plot(x, d3y, "b.")

## Problema 5

A técnica básica para resolver equações não lineares do tipo

$$
f(x) = 0
$$

é o método de Newton-Raphson. Neste método, a partir de um chute inicial $x_0$, é feita uma estimativa $x_1$, em seguida, $x_2$, $\ldots$, $\x_n$ e se tivermos sorte este processo converge para uma solução do problema.

Como $x_0$ não é solução do problema, queremos encontrar $\Delta x$ que forneça uma estimativa melhor da solução:

$$
f(x + \Delta x) = 0 \:\Longrightarrow\: f(x+\Delta x) \approx f(x) + \Delta x\cdot f'(x) = 0 \:\Longrightarrow\: \Delta x = -\frac{f(x)}{f'(x)}
$$


Escreva uma função `newtonraphson` que recebe uma função e um chute inicial para encontrar um zero.

In [ ]:
using ForwardDiff

In [ ]:
function newtonraphson(f, df, x0, err=1e-10, maxiter=100)
    niter = 0
    for i = 1:maxiter
        dx = f(x0) / df(x0)
        x0 -= dx
        niter = i
        if abs(dx) < err
            break
        end
    end
    
    return x0, niter
end


In [ ]:
function plotnewton(f, df, x0, a, b, err=1e-5, maxiter=10)
    nn = 200
    xx = range(a,b, length=nn)
    yy = f.(xx)
    
    plot(xx, yy)
    axhline("k")
    for i = 1:maxiter
        y = f(x0)
        dy = df(x0)
        dx = y / dy
        plot([x0, x0], [0.0, y], "k:")
        plot([x0, x0-dx], [y, 0.0], "k-")
        x0 = x0 - dx
    end
        
        
end

In [ ]:
# Raíz quadrada de 2
a = 2.0
g(x) = x*x-a
dg(x) = 2x
newtonraphson(g, dg, 1.0)

In [ ]:
plotnewton(g, dg, 2, 1.3, 2.0)

#### Usando ForwardDiff

In [ ]:

function newtonraphson2(f, x0, err=1e-10, maxiter=100)
    niter = 0
    for i = 1:maxiter
        dx = f(x0) / ForwardDiff.derivative(f,x0)
        x0 -= dx
        niter = i
        if abs(dx) < err
            break
        end
    end
    
    return x0, niter
    

end

In [ ]:
newtonraphson2(g, 2.0)